# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [33]:
type(df['length'][0])

numpy.float64

In [31]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/yacinus/Desktop/cassandra-project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [9]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [10]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS spokifydb
                    WITH REPLICATION = 
                    {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [11]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('spokifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [19]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS song_library "
query1 += "(sessionId int, itemInSession int,artist text,song text,length float,PRIMARY KEY (sessionId, itemInSession))"
    

def create_tbl(session,query):
    try:
        session.execute(query)
    except Exception as e:
        print(e)

create_tbl(session, query1)    

In [23]:
query2 = "CREATE TABLE IF NOT EXISTS artist_user_library "
query2 += "(itemInSession int, artist text, song text, firstName text, lastName text, userId int, sessionId int,PRIMARY KEY(userId,sessionId,itemInSession))"
 
create_tbl(session, query2)

In [24]:
query3 = "CREATE TABLE IF NOT EXISTS user_per_song "
query3 += "(firstName text, lastName text, song text, PRIMARY KEY(song))"

create_tbl(session, query3)

In [34]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Insert data into song_library
        query = "INSERT INTO song_library (sessionId,itemInSession,artist,song,length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))
        #Insert data into artist_user_library
        
        
['artist':0,'firstName':1,'gender':2,'itemInSession':3,'lastName':4,'length':5,\
                'level':6,'location':7,'sessionId':8,'song':9,'userId':10])


In [ ]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Insert data into song_library
        query1 = "INSERT INTO artist_user_library (itemInSession,artist,song,firstName,lastName,userId,sessionId)"
        query1 = query1 + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query1, (int(line[3]), line[0], line[9], line[1],line[4], int(line[10]), int(line[8])))
    

In [44]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Insert data into song_library
        query2 = "INSERT INTO user_per_song (firstName,lastName,song)"
        query2 = query2 + "VALUES (%s,%s,%s)"
        session.execute(query2, (line[1],line[4],line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


                    

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

In [43]:
os.path.abspath(os.path.join(os.getcwd()) + '/input/event_data')

'/home/yacinus/Desktop/cassandra-project/input/event_data'

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [45]:
tables = ['user_per_song', 'artist_user_library', 'song_library']
for t in tables:
    session.execute("DROP TABLE {}".format(t))

### Close the session and cluster connection¶

In [46]:
session.shutdown()
cluster.shutdown()

In [26]:
os.path.dirname(os.path.abspath('input'))

'/home/yacinus/Desktop/cassandra-project'

In [44]:
df = pd.read_csv('event_datafile_new.csv')

In [50]:
type(df['length'][0])

numpy.float64

In [51]:
def insert_rows_from_df(cur, df_name, insert_query, cols):
    for i, row in df_name.iterrows():
        print(row)

In [58]:
dg = df.iloc[:2]

In [73]:
ll = [0,1,2,3]
for i, row in dg.iterrows():
        print(i, (*[row[x] for x in ll]))
        print('-------')

0 Stephen Lynch Jayden M 0
-------
1 Manowar Jacob M 0
-------


In [65]:
dg.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [76]:
[*[[x] for x in ll]]

[[0], [1], [2], [3]]

In [78]:
insert_table_queries = [('a', [3, 0, 9, 1, 4, 10, 8]),
                        ('b', [8, 3, 0, 9, 5]), ('c', [1, 4, 9])]


In [80]:
for x in insert_table_queries:
    print(x[0],(*x[1]))

a 3 0 9 1 4 10 8
b 8 3 0 9 5
c 1 4 9


In [81]:
df[1:5]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


In [82]:
ll

[0, 1, 2, 3]

In [83]:
def row(x):
    return x*2

In [84]:
a = ()
a = (*[row[x] for x in ])

SyntaxError: can't use starred expression here (<ipython-input-84-74c58e1e7102>, line 5)